## Introduction

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

**How to Train [Detectron2](https://github.com/facebookresearch/detectron2) Segmentation on a Custom Dataset**

The notebook is based on official Detectron2 [colab notebook](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5) and it covers:
- Python environment setup
- Inference using pre-trained models
- Download, register and visualize COCO Format Dataset
- Configure, train and evaluate model using custom COCO Format Dataset

**Preparing a Custom Dataset**

In this tutorial, we will utilize an open source computer vision dataset from one of the 100,000+ available on [Roboflow Universe](https://universe.roboflow.com).

If you already have your own images (and, optionally, annotations), you can convert your dataset using [Roboflow](https://roboflow.com), a set of tools developers use to build better computer vision models quickly and accurately. 150k+ developers use roboflow for (automatic) annotation, converting dataset formats (like to Detectron2), training, deploying, and improving their datasets/models.

Follow [the getting started guide here](https://docs.roboflow.com/quick-start) to create and prepare your own custom dataset. Make sure to select **Instance Segmentation** Option, If you want to create your own dataset on roboflow

Useful Dataset Links

* [Helmet Instace Segmentation ](https://universe.roboflow.com/computer-vision-hx9i9/helmet_polygon_v2/dataset/4)

* [PCB Board Instance Segmentation](https://universe.roboflow.com/chip/pcb_segmentation_yolov7/dataset/17)

* [Fire Segmentation Instance Segmentation](https://universe.roboflow.com/fire-instance-segmentation/fire-detection-pr6nj/dataset/1)

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator` and set it to `GPU`.

## Install Detectron2 and dependencies

Now is a good time to confirm that we have the right versions of the libraries at our disposal.

In [34]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

zsh:1: command not found: nvcc
torch:  2.1 ; cuda:  2.1.2
detectron2: 0.6


In [35]:
# COMMON LIBRARIES
import os
import cv2
import torchvision
from datetime import datetime
#from google.colab.patches import cv2_imshow

# DATA SET PREPARATION AND LOADING
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

# VISUALIZATION
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# CONFIGURATION
from detectron2 import model_zoo
from detectron2.config import get_cfg

# EVALUATION
from detectron2.engine import DefaultPredictor

# TRAINING
from detectron2.engine import DefaultTrainer

## COCO Format Dataset

### Download

We use `football-pitch-segmentation` dataset as example. Feel free to visit [Roboflow Universe](https://universe.roboflow.com/), and select any other Instance Segmentation dataset. Make sure to download the dataset in correct - `COCO Segmentation` format.

Structure of your dataset should look like this:

```
dataset-directory/
├─ README.dataset.txt
├─ README.roboflow.txt
├─ train
│  ├─ train-image-1.jpg
│  ├─ train-image-1.jpg
│  ├─ ...
│  └─ _annotations.coco.json
├─ test
│  ├─ test-image-1.jpg
│  ├─ test-image-1.jpg
│  ├─ ...
│  └─ _annotations.coco.json
└─ valid
   ├─ valid-image-1.jpg
   ├─ valid-image-1.jpg
   ├─ ...
   └─ _annotations.coco.json
```

In [36]:
#!pip install roboflow
'''

from roboflow import Roboflow
rf = Roboflow(api_key="WE1IzumYGoc2pjeOHe5g")
project = rf.workspace("het-m73ej").project("bilgi-university-car-damage")
dataset = project.version(2).download("coco-segmentation")
'''
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="WE1IzumYGoc2pjeOHe5g")
project = rf.workspace("het-m73ej").project("bilgi-university-car-damage")
dataset = project.version(1).download("coco-segmentation")


'\n\nfrom roboflow import Roboflow\nrf = Roboflow(api_key="WE1IzumYGoc2pjeOHe5g")\nproject = rf.workspace("het-m73ej").project("bilgi-university-car-damage")\ndataset = project.version(2).download("coco-segmentation")\n'

### Register

When you use Detectron2, before you actually train the model you need to [register it](https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html#register-a-coco-format-dataset).

In [37]:
DATA_SET_NAME = 'Bilgi-University-Car-Damage'
ANNOTATIONS_FILE_NAME = "_annotations.coco.json"

In [38]:
# TRAIN SET
TRAIN_DATA_SET_NAME = f"{DATA_SET_NAME}-train"
#TRAIN_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "train")
#TRAIN_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "train", ANNOTATIONS_FILE_NAME)

#register_coco_instances(
 #   name=TRAIN_DATA_SET_NAME,
 #   metadata={},
 #   json_file=TRAIN_DATA_SET_ANN_FILE_PATH,
 #   image_root=TRAIN_DATA_SET_IMAGES_DIR_PATH
#)

# TEST SET
TEST_DATA_SET_NAME = f"{DATA_SET_NAME}-test"
#TEST_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "test")
#TEST_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "test", ANNOTATIONS_FILE_NAME)

#register_coco_instances(
 #   name=TEST_DATA_SET_NAME,
 #   metadata={},
 #   json_file=TEST_DATA_SET_ANN_FILE_PATH,
 #   image_root=TEST_DATA_SET_IMAGES_DIR_PATH
#)
'''
# VALID SET
VALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"
VALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")
VALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)

register_coco_instances(
    name=VALID_DATA_SET_NAME,
    metadata={},
    json_file=VALID_DATA_SET_ANN_FILE_PATH,
    image_root=VALID_DATA_SET_IMAGES_DIR_PATH
)
'''

'\n# VALID SET\nVALID_DATA_SET_NAME = f"{DATA_SET_NAME}-valid"\nVALID_DATA_SET_IMAGES_DIR_PATH = os.path.join(dataset.location, "valid")\nVALID_DATA_SET_ANN_FILE_PATH = os.path.join(dataset.location, "valid", ANNOTATIONS_FILE_NAME)\n\nregister_coco_instances(\n    name=VALID_DATA_SET_NAME,\n    metadata={},\n    json_file=VALID_DATA_SET_ANN_FILE_PATH,\n    image_root=VALID_DATA_SET_IMAGES_DIR_PATH\n)\n'

We can now confirm that our custom dataset was correctly registered using [MetadataCatalog](https://detectron2.readthedocs.io/en/latest/modules/data.html#detectron2.data.MetadataCatalog).

In [39]:
[
    data_set
    for data_set
    in MetadataCatalog.list()
    if data_set.startswith(DATA_SET_NAME)
]

['Bilgi-University-Car-Damage-train',
 'Bilgi-University-Car-Damage-test',
 'Bilgi-University-Car-Damage-valid']

### Visualize

Let's take a look at single entry from out train dataset.

In [40]:
metadata = MetadataCatalog.get(DATA_SET_NAME + "-test")

'''
metadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)
dataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)

dataset_entry = dataset_train[0]
image = cv2.imread(dataset_entry["file_name"])

visualizer = Visualizer(
    image[:, :, ::-1],
    metadata=metadata,
    scale=0.8,
    instance_mode=ColorMode.IMAGE_BW
)

out = visualizer.draw_dataset_dict(dataset_entry)
cv2.imshow("enes",out.get_image()[:, :, ::-1])

cv2.waitKey(0)   #wait for a keyboard input
cv2.destroyAllWindows()
'''

'\nmetadata = MetadataCatalog.get(TRAIN_DATA_SET_NAME)\ndataset_train = DatasetCatalog.get(TRAIN_DATA_SET_NAME)\n\ndataset_entry = dataset_train[0]\nimage = cv2.imread(dataset_entry["file_name"])\n\nvisualizer = Visualizer(\n    image[:, :, ::-1],\n    metadata=metadata,\n    scale=0.8,\n    instance_mode=ColorMode.IMAGE_BW\n)\n\nout = visualizer.draw_dataset_dict(dataset_entry)\ncv2.imshow("enes",out.get_image()[:, :, ::-1])\n\ncv2.waitKey(0)   #wait for a keyboard input\ncv2.destroyAllWindows()\n'

## Train Model Using Custom COCO Format Dataset

### Configuration

In [41]:
import os
# HYPERPARAMETERS
ARCHITECTURE = "mask_rcnn_R_101_FPN_3x" #mask_rcnn_X_101_32x8d_FPN_3x
CONFIG_FILE_PATH = f"COCO-InstanceSegmentation/{ARCHITECTURE}.yaml"
MAX_ITER = 4500
EVAL_PERIOD = 350
BASE_LR = 0.0001
NUM_CLASSES = 8

# OUTPUT DIR
OUTPUT_DIR_PATH = os.path.join(
    DATA_SET_NAME,
    ARCHITECTURE,
    datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
)

os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)

In [42]:
from detectron2.config import get_cfg
cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"
cfg.merge_from_file(model_zoo.get_config_file(CONFIG_FILE_PATH))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(CONFIG_FILE_PATH)
cfg.DATASETS.TRAIN = (TRAIN_DATA_SET_NAME,)
cfg.DATASETS.TEST = (TEST_DATA_SET_NAME,)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.TEST.EVAL_PERIOD = EVAL_PERIOD
cfg.DATALOADER.NUM_WORKERS = 4
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
cfg.OUTPUT_DIR = OUTPUT_DIR_PATH

### Training

In [43]:
'''
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
'''

'\n\ntrainer = DefaultTrainer(cfg)\ntrainer.resume_or_load(resume=False)\ntrainer.train()\n'

In [44]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir $OUTPUT_DIR_PATH

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

### Evaluation

In [74]:
cfg.MODEL.WEIGHTS =  "model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
predictor = DefaultPredictor(cfg)

In [75]:
'''

dataset_valid = DatasetCatalog.get(VALID_DATA_SET_NAME)

for d in dataset_valid:
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)

    visualizer = Visualizer(
        img[:, :, ::-1],
        metadata=metadata,
        scale=0.8,
        instance_mode=ColorMode.IMAGE
    )
    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("car",out.get_image()[:, :, ::-1])
    
    cv2.waitKey(0)   #wait for a keyboard input
    cv2.destroyAllWindows()
    '''

'\n\ndataset_valid = DatasetCatalog.get(VALID_DATA_SET_NAME)\n\nfor d in dataset_valid:\n    img = cv2.imread(d["file_name"])\n    outputs = predictor(img)\n\n    visualizer = Visualizer(\n        img[:, :, ::-1],\n        metadata=metadata,\n        scale=0.8,\n        instance_mode=ColorMode.IMAGE\n    )\n    out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))\n    cv2.imshow("car",out.get_image()[:, :, ::-1])\n    \n    cv2.waitKey(0)   #wait for a keyboard input\n    cv2.destroyAllWindows()\n    '

In [76]:
'''

from detectron2.utils.visualizer import ColorMode
import glob
#from google.colab.patches import cv2_imshow
for imageName in glob.glob('/Users/het/PycharmProjects/finalProject/1.jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=metadata,
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2.imshow("car",out.get_image()[:, :, ::-1])
  cv2.waitKey(0)   #wait for a keyboard input
  cv2.destroyAllWindows()
  
  '''

'\n\nfrom detectron2.utils.visualizer import ColorMode\nimport glob\n#from google.colab.patches import cv2_imshow\nfor imageName in glob.glob(\'/Users/het/PycharmProjects/finalProject/1.jpg\'):\n  im = cv2.imread(imageName)\n  outputs = predictor(im)\n  v = Visualizer(im[:, :, ::-1],\n                metadata=metadata,\n                scale=0.8\n                 )\n  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))\n  cv2.imshow("car",out.get_image()[:, :, ::-1])\n  cv2.waitKey(0)   #wait for a keyboard input\n  cv2.destroyAllWindows()\n  \n  '

In [77]:
import cv2
# Load an image from a file
image_path = "/Users/het/PycharmProjects/finalProject/1.jpg"
img = cv2.imread(image_path)

# Make a prediction using the model
outputs = predictor(img)

# Visualize the prediction
visualizer = Visualizer(
    img[:, :, ::-1],
    metadata=metadata,
    scale=0.8,
    instance_mode=ColorMode.IMAGE
)
out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("car",out.get_image()[:, :, ::-1])
while True:
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == ord('Q'):
        break

#Destroy OpenCV window
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

TypeError: 'NoneType' object is not subscriptable

In [55]:
outputs["instances"].pred_classes

tensor([5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])